In [ ]:
#!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv torch_geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

## Cora citation network

To demonstrate, we make use of the `Cora` dataset, which is a **citation network** where nodes represent documents.
Each node is described by a 1433-dimensional bag-of-words feature vector.
Two documents are connected if there exists a citation link between them.  
The task are:
- Node classification: to infer the category of each document (7 in total).
- Link prediction: to predict missing citations between documents.

This dataset was first introduced by [Yang et al. (2016)](https://arxiv.org/abs/1603.08861) as one of the datasets of the `Planetoid` benchmark suite.

In [ ]:
import numpy as np

import torch
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='../data/Planetoid', name='Cora')

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object

In [ ]:
data

In [ ]:
# x : node feature matrix [#num_nodes, #num_features] 
# edge_index :  [#nodes_in_edge, #num_edges] e.g [[1,2],[3,4]] => (1,3), (2,4) in E
# y : labels
# train_mask, val_mask, test_mask : bit mask1

Data fields are not fixed, you can add/delete fields

In [ ]:
data.z = torch.Tensor(np.array([0 for i in range(len(data.x))]))
data

In [ ]:
del(data.z)

In [ ]:
data

We delete masks to perform custom nodes split

In [ ]:
del(data.train_mask)
del(data.val_mask)
del(data.test_mask)

In [ ]:
# 1 - Normalize features

from torch_geometric.transforms import NormalizeFeatures

transform = NormalizeFeatures() #Row-normalizes the attributes to sum-up to one
data = transform(data)

In [ ]:
# 2 - Random Node Split

from torch_geometric.transforms import RandomNodeSplit

split = RandomNodeSplit(num_val=0.0,num_test=0.30)
data = split(data)

In [ ]:
print(data)
print(f'Training set rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Validation set rate: {int(data.val_mask.sum()) / data.num_nodes:.2f}')
print(f'Test set rate: {int(data.test_mask.sum()) / data.num_nodes:.2f}')

## Load your own dataset

Many options...  
My favourite is use `from\_networkx` util 

In [ ]:
import networkx as nx

mouse = nx.read_edgelist('../data/bn-mouse-kasthuri_graph_v4.edges')

In [ ]:
#Do everything you can on networkx (add attributes, compute features, remove self-loops / isolated nodes, ...)

#Example: add degree as node feature
deg = dict(mouse.degree())
nx.set_node_attributes(mouse,deg,'degree')

In [ ]:
from torch_geometric.utils import from_networkx

mouse_data = from_networkx(mouse, group_node_attrs = all)

In [ ]:
mouse_data

#### No feature? No problem! Constant encoder

In [ ]:
from torch_geometric.transforms import Constant

constant = Constant(cat=False) #cat = False drop the other features
mouse_data = constant(mouse_data)

In [ ]:
mouse_data.x

#### Create your own dataset in PyG: https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data

## Node2Vec

Let's see together the example available on pyg repository: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/node2vec.py . To understand its parameters you can refer to: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.Node2Vec.html#torch_geometric.nn.models.Node2Vec

## GNN Model

In [ ]:
from torch_geometric.nn import GCNConv, Linear
import torch.nn.functional as F

In [ ]:
class CoraGCN(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        torch.manual_seed(1234567) #manual seed for reproducibility
        
        self.lin1 = Linear(dataset.num_features, 64) 
        self.lin2 = Linear(64, 32) #two MLP layers to preprocess the BoW
        self.conv1 = GCNConv(32, 16) 
        self.conv2 = GCNConv(16, dataset.num_classes) #two GCNConv to compute embeddings
        
    def reset_parameters(self):
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x, edge_index):
        #x : node feature matrix, edge_index : structure of the graph
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.lin2(x)
        x = x.relu()
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x, edge_index)
        
        return x

### Training

In [ ]:
model = CoraGCN()
model.reset_parameters()

In [ ]:
model

<img src="../img/CoraGCN.png" alt="nodepred_gcn" width="500"/>

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4) #SGD, Adagrad
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    
    ### NODE CLASSIFICATION ACTS HERE ###
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    ### --- ###
    
    # Evaluation
    train_correct = pred[data.train_mask] == data.y[data.train_mask]  # Check against ground-truth labels for train set.
    train_acc = int(train_correct.sum()) / int(data.train_mask.sum())  # Derive ratio of correct predictions for train set.
    test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels for test set.
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions for test set.
    return train_acc, test_acc


for epoch in range(1, 101):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
train_acc,test_acc = test()
print(f'Train Accuracy: {train_acc:.4f}\nTest Accuracy: {test_acc:.4f}')

In [ ]:
# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy()) #TSNE applied to node embeddings as numpy arrays

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

model.eval()
out = model(data.x, data.edge_index)
visualize(out, color=data.y)

### Exercise 0 - GNN without G

What is the effect of performing GCN forward computation using an empty edge index? First, try to answer without run the experiment. Then, try to reproduce your intuition.

### Exercise 1 - CoraGAT

Write a GNN-based architecture for solving Node Classification task on Cora with the following requirements:
- Use two MLP layers to pre-process BoW representations. The first MLP layer has 1433 input channels and 128 output channels. The second MLP layer has (128, 64) channels.
- Use two graph attention network layers to obtain node embeddings. The graph attentional operator from the "Graph Attention Networks" is available in torch_geometric.nn as GATConv. The GAT layers have respectively (64,32) and (32,16) channels.
- Use ReLu as activation function and apply dropout on the hidden layers.
- Use a Linear layer to post-process the node embeddings and perform the decoder step. The Linear layer has (16,num_classes) channels.
- Use a log softmax activation function on the output layer. Log softmax is available in torch.nn.functional.log_softmax.


### Exercise 2 - Dive into evaluation

Compare CoraGCN and CoraGAT on the node classification task.  
Important: the node split masks as well as the weights' initialization into the GNN models must be the same.  
Perform also this additional steps:

- Conduct some hyperparameter optimization on Adam hypeparameters
- Print train and test accuracy along the epochs
- Print different evaluation metrics (e.g. confusion matrix)

### Link Prediction

#### Link split and negative sampling

In [ ]:
#Perform random link split
from torch_geometric.transforms import RandomLinkSplit

link_split = RandomLinkSplit(num_val=0.0,num_test=0.25)
train_link, val_link, test_link = link_split(data)

In [ ]:
train_link

In [ ]:
# edge_label : 1 for closed link, 0 for open link
# edge_label_index: edge_index + negative_sampling edge_index

`RandomLinkSplit` performs split + negative sampling: this is perfect for static networks. Suppose you want to test your model in a future time interval... you do not need to perform a link split. How to perform negative sampling?

In [ ]:
from torch_geometric.utils import negative_sampling

future_data = Planetoid(root='data/Planetoid', name='Cora')[0]

#NEGATIVE SAMPLING
future_neg_edge_index = negative_sampling(
        edge_index=future_data.edge_index, #positive edges
        num_nodes=future_data.num_nodes, # number of nodes
        num_neg_samples=future_data.edge_index.size(1)) # number of neg_sample equal to number of pos_edges

In [ ]:
future_neg_edge_index

In [ ]:
#edge index ok, edge_label concat, edge_label_index concat
num_pos_edge = future_data.edge_index.size(1)
future_data.edge_label = torch.Tensor(np.array([1 for i in range(num_pos_edge)] + [0 for i in range(num_pos_edge)]))
future_data.edge_label_index = torch.cat([future_data.edge_index, future_neg_edge_index], dim=-1)

In [ ]:
future_data

#### GNN Training

In [ ]:
from sklearn.metrics import roc_auc_score

We can re-use the same GNN model!

<img src="../img/CoraGCNLinkPre.png" alt="linkpred_gcn" width="500"/>

In [ ]:
model = CoraGCN()
model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion =  torch.nn.BCEWithLogitsLoss() #change loss function

def train_linkpre():
    
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(train_link.x, train_link.edge_index)  # Perform a single forward pass.
    
    ### LINK PREDICTION ACTS HERE ###
    
    out_src = out[train_link.edge_label_index[0]] #embedding src nodes
    out_dst = out[train_link.edge_label_index[1]] #embedding dst nodes
    
    # LINK EMBEDDING #
    # 1 - Dot Product
    out_sim = out_src * out_dst #dotproduct
    pred = torch.sum(out_sim, dim=-1)
    
    # 2 - Concatenation + linear function
    #out_sim = torch.cat([out_src, out_dst], dim=-1)
    #pred = torch.sum(out_sim,dim=-1)
    
    
    loss = criterion(pred, train_link.edge_label.type_as(pred))  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test_linkpre(test_link):
    model.eval()
    out = model(test_link.x, test_link.edge_index)
    
    ### LINK PREDICTION ACTS HERE ###
    
    out_src = out[test_link.edge_label_index[0]] #embedding src nodes
    out_dst = out[test_link.edge_label_index[1]] #embedding dst nodes
    
    # LINK EMBEDDING #
    # 1 - Dot Product
    out_sim = out_src * out_dst
    h = torch.sum(out_sim, dim=-1)
    
    # 2 - Concatenation + linear function
    #out_sim = torch.cat([out_src, out_dst], dim=-1)
    #h = torch.sum(out_sim,dim=-1)
    
    pred_cont = torch.sigmoid(h).cpu().detach().numpy()
    
    # EVALUATION
    test_label = test_link.edge_label.cpu().detach().numpy() #retrieve test set labels
    test_roc_score = roc_auc_score(test_label, pred_cont) #comput AUROC score for test set
    
    return test_roc_score


for epoch in range(1, 101):
    loss = train_linkpre()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
roc_train = test_linkpre(train_link)
roc_test = test_linkpre(test_link)
print(f'Train AUROC: {roc_train:.4f}\nTest AUROC: {roc_test:.4f}')

### Exercise 3 - HadamardMLP
Actually, there are two popular solutions to perform the decoder step of link prediction tasks: the dot product and the HadamardMLP architecture. After theoretical and empirical analysis, [Wang et al. (2022)](https://arxiv.org/abs/2209.10100) find that the HadamardMLP decoders are generally more effective for LP.  

Implement the HadamardMLP decoder into the CoraGCN architecture. The HadamardMLP consists of two steps:
- Perform the hadamard product between the embeddings of nodes src and the embeddings of nodes dst.
- Use a MLP layer to post-process the hadamard product.  

Note that, for this task, you need to create a CoraGCNLP class that returns link prediction scores as the output of the forward method. Only in this way, the MLP layer parameters can be updated through the backpropagation steps.